In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="5" color="red">ch09_파일 입출력(i/o) 프로그래밍</font></b>

1. *.txt\
 김회원, 33, 아무동9\
 이회원, 22, 무시기동8

2. pickle파일(*.data) : 객체리스트 저장, 머신러닝모델 저장용, 메모장으로 열 수 없음

3. *.csv\
   name,age,address\
   "김회원",33,"아무동"\
   "이회원",22,"무시기동8"\
   단, 항목 사이에 스페이스 없어야함, ""은 스트링, 없을땐 숫자로 인식
   
4. *.json\
   실시간의 데이터를 구하고 싶을 때, 특정 API를 사용하게 되는데 그때 json파일로 많이 받는다.\
   [{'name':'김회원', 'age':33, 'address':'아무동9'}, \
    {'name':'이회원', 'age':22, 'address':'무시기동8'}]
    
5. *.h5 이나 *.hdf5  (=> 딥러닝 모델 저장)\
   HDVview로 읽을 수 있음

# 1절. txt 파일에 데이터 저장하고 불러오기

In [ ]:
# 지난시간 예외처리 방법
try:
    f = open('data/ch09.txt', 'w')
    # 데이터 출력
except FileNotFoundError as e:
    print(e)
finally:
    f.close()


In [2]:
#수업을 위해 간단히 
f = open('data/ch09.txt', 'w')
print('쓰기가능한지 여부 :', f.writable())

쓰기가능한지 여부 : True


In [3]:
f.write('Hello\nWorld')
f.close()  # close() 쓰지않으면 저장되지 않음

In [5]:
#close 조차 사용하지 않게 with 절 사용
with open('data/ch09.txt', 'w') as f:
    print('쓰기가능한지 여부 :', f.writable())
    f.write('Hello\nPython')

쓰기가능한지 여부 : True


In [11]:
# mode 
    # 'r' or 'rt' : text읽기 모드
    # 'rb' : 바이너리 읽기 모드
    # 'w' or 'wt' : text쓰기 모드 (파일이 있으면 덮어쓰기, 없으면 생성 후 쓰기)
    # 'wb' : 바이너리 쓰기 모드
    # 'a' or 'at': text 추가(append) 모드 (파일이 있으면 추가, 없으면 생성 후 쓰기)
    # 읽기모드에서는 '파일'이 없으면 예외/쓰기모드에서는 '폴더'가 없으면 예외
# 한글을 쓸때는 encoding
    # euc-kr (한글완성형) 믜x
    # cp949 (확장된 한글완성형) : open() 함수 기본 encoding방식 (win)
    # utf-8 (한글조합형) : open() 함수 기본 encoding방식 (mac, linux), 주피터노트북 기본
with open('data/ch09.txt', 'w') as f:
    print('쓰기가능한지 여부 :', f.writable())
    f.write('Hello\nPython')

쓰기가능한지 여부 : True


In [15]:
with open('data/ch09.txt', 'w', encoding='utf-8') as f:
    # 1방법
    f.write('홍길동, 33, 아무동9\n')
    f.write('김길동, 35, 무개동9\n')

    # 2방법
    textlist = ['홍길동, 33, 아무동9\n', '김길동, 35, 무개동9\n']
    for line in textlist:
        f.write(line)

    # 3방법
    f.writelines(textlist)

In [19]:
# 한줄씩 읽기 
with open('data/ch09.txt', 'r', encoding='cp949') as f:
    line = f.readline()
    while line!='':
        print(line, end='')
        line = f.readline()

홍길동, 33, 아무동9
김길동, 35, 무개동9
홍길동, 33, 아무동9
김길동, 35, 무개동9
홍길동, 33, 아무동9
김길동, 35, 무개동9


In [20]:
# 모든 줄을 읽기
with open('data/ch09.txt', 'r') as f:
    lines = f.readlines()
    print(lines)

['홍길동, 33, 아무동9\n', '김길동, 35, 무개동9\n', '홍길동, 33, 아무동9\n', '김길동, 35, 무개동9\n', '홍길동, 33, 아무동9\n', '김길동, 35, 무개동9\n']


In [23]:
# 모두 읽기
with open('data/ch09.txt', 'r') as f:
    lines = f.read()
    print(lines)

홍길동, 33, 아무동9
김길동, 35, 무개동9
홍길동, 33, 아무동9
김길동, 35, 무개동9
홍길동, 33, 아무동9
김길동, 35, 무개동9



# 2절. 피클 파일 저장/불러오기
- 객체 혹은 객체리스트로 저장. 즉 클래스를 사용

## 2.1 형식이 있는 txt데이터 불러오기

In [29]:
with open('data/ch09_member.txt', 'r') as f:
    line = f.readline()
    print(line)

UnicodeDecodeError: 'cp949' codec can't decode byte 0xed in position 0: illegal multibyte sequence

In [47]:
class Member:
    def __init__(self, name, age, email, address):
        self.name = name
        self.age = age
        self.email = email
        self.address = address
    def __str__(self):
        return '{}, {}, {}, {}'.format(self.name,
                                       '성년' if self.age >18 else '미성년',
                                       self.email,
                                       self.address)
    def as_dict(self):
        return {
            'name':self.name,
            'age':self.age,
            'email':self.email,
            'address':self.address
                }    
    ## 객체간 같은 내용이면 True 값을 내기위해 아래 내용으로 인스턴스 함수 추가
    def __eq__(self, other):
        return self.name == other.name and\
            self.age == other.age and \
            self.email==other.email and\
            self.address==other.address

In [35]:
user1 = Member('홍',20,'a@a.com','신림동')
user2 = Member('홍',20,'a@a.com','신림동')
print(user1==user2)    #  객체간 == 은 __eq__ 를 내부적으로 살펴본다.
print(user1.__eq__(user2))   # 같은 내용이면 True로 하고 싶다

True
True


In [48]:
# 파일을 읽어서 한줄씩 읽고
# 형식이 있는 txt 파일 내용을 member list(피클 저장), 딕셔너리 list(데이터프레임) 저장
user_list = []  # 객체 리스트로 구현
user_dict = []  # 딕셔너리 리스트로 구현

with open('data/ch09_member.txt', 'r', encoding='utf-8') as txt_file:
    lines = txt_file.readlines()
# print(lines)

for line in lines:
    data = line.split(',')
    #print(data)
    name = data[0]
    age = int(data[1].strip()) # strip() : 좌우 공백(spac, \t, \n) 제거 
    email = data[2].strip()
    address = data[3].strip()
    user = Member(name, age, email, address)
    user_list.append(user)
    user_dict.append(user.as_dict())
    # user_dict.append(user.__dict__)
              

In [49]:
for user in user_list:
    print(user)

홍길동, 성년, kildong@hong.com, 서울시 관악구
홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
신길동, 성년, shinkil@hong.com, 서울시 동작구


In [56]:
print(user_dict)

[{'name': '홍길동', 'age': 20, 'email': 'kildong@hong.com', 'address': '서울시 관악구'}, {'name': '홍길숙', 'age': 25, 'email': 'kilsuk1@hong.com', 'address': '서울시 영등포구'}, {'name': '신길동', 'age': 30, 'email': 'shinkil@hong.com', 'address': '서울시 동작구'}]


## 2.2 피클링
- 객체 리스트(user_list) -> 피클 파일로 쓰기
- 피클 파일을 읽기 -> 객체 리스트(load_user_list)
- 저장 : pickle.dump(객체명, 스트림명)
- 로드 : 로드할객체명 = pickle.load(스트림명)

In [51]:
# 피클링을 이용한 객체를 저장하기, 바이너리 모드로 써야한다
import pickle
with open('data/ch09_member.data', 'wb') as f:
    pickle.dump(user_list, f)

In [61]:
# 피클링을 이용한 파일에서 객체 데이터로 읽기
with open('data/ch09_member.data', 'rb') as f:
    load_user_list = pickle.load(f)

for data in load_user_list:
    print(data)

홍길동, 성년, kildong@hong.com, 서울시 관악구
홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
신길동, 성년, shinkil@hong.com, 서울시 동작구


In [60]:
load_user_list==user_list

True

In [66]:
# 입력한 객체리스트와 로드한 객체리스트의 각 항목 값이 같은지 출력하고 확인하는 구문
for idx, (user, load_user) in enumerate(zip(user_list, load_user_list)):
    print(idx, user)
    print(idx, load_user)
    print(user==load_user)

0 홍길동, 성년, kildong@hong.com, 서울시 관악구
0 홍길동, 성년, kildong@hong.com, 서울시 관악구
True
1 홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
1 홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
True
2 신길동, 성년, shinkil@hong.com, 서울시 동작구
2 신길동, 성년, shinkil@hong.com, 서울시 동작구
True


In [70]:
# 방대한 양을 일일이 확인할 수는 없다. 

result = []
for user, load_user in zip(user_list, load_user_list):
    result.append(user==load_user)
print(len(set(result))==1)
all(result)     # all() : 모두의 값이 같은지 체크하는 함수

True


True

# 3절. csv파일 읽기/쓰기
- 헤더가 없는 경우 : 헤더를 만들어줘야한다
- csv파일 <-> 리스트 데이터    3.1 읽기 / 3.2 쓰기
- csv파일 <-> 딕셔너리 리스트   3.3 읽기 / 3.4 쓰기

## 3.1 reader - csv파일 리스트로 읽기 

In [77]:
import csv
with open('data/ch09_member1.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)  
    # csv.변수명(스트림명) : 한줄씩 읽어들여 리스트로 만드는 것을 반복하는 함수
    result = list(reader)   # with절이 끝나면 날아가므로 결과를 저장할 필요가 있어서 result에 저장
print(result)

[['홍길동', '20', 'kildong@hong.com', '서울시 관악구'], ['김길동', '40', 'kimdong@hong.com', '서울시 영등포구'], ['신길동', '30', 'sindong@hong.com', '서울시 동작구']]


In [78]:
# ""(따옴표)가 없는 데이터는 numeric으로 csv.QUOTE_NONNUMERIC <단 실수로 나옴
import csv
with open('data/ch09_member1.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f,
                       quoting=csv.QUOTE_NONNUMERIC)  
    result = list(reader)
print(result)

[['홍길동', 20.0, 'kildong@hong.com', '서울시 관악구'], ['김길동', 40.0, 'kimdong@hong.com', '서울시 영등포구'], ['신길동', 30.0, 'sindong@hong.com', '서울시 동작구']]


In [81]:
# 이 리스트, 딕셔너리로 바꿔봅시다
import csv
with open('data/ch09_member1.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f,
                       quoting=csv.QUOTE_NONNUMERIC)  
    result = list(reader)
dict_list = []
for data in result:
    dict_list.append({
        'name':data[0],
        'age':int(data[1]),
        'email':data[2],
        'address':data[3]
    })
# 길다. 하기 싫다. 간편하게 처리하는건 3.3에서 배울 예정.
# 질문. 해당 데이터에 헤더가 있었다면, __dict__ 쓸수 있었던건가?

print(dict_list)

[{'name': '홍길동', 'age': 20, 'email': 'kildong@hong.com', 'address': '서울시 관악구'}, {'name': '김길동', 'age': 40, 'email': 'kimdong@hong.com', 'address': '서울시 영등포구'}, {'name': '신길동', 'age': 30, 'email': 'sindong@hong.com', 'address': '서울시 동작구'}]


## 3.2 writer

In [82]:
user_list = [['홍길동', 20, 'kildong@hong.com', '서울시 관악구'],
            ['김길동', 40, 'kildong@kim.com', '서울시 강서']]

In [89]:
with open('data/ch09_member1_write.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(user_list)

In [87]:
with open('data/ch09_member1_write.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f, quoting=csv.QUOTE_NONNUMERIC)  # ""있는 건 있는걸로 ""없는 건 없는걸로 write
    writer.writerows(user_list)

## 3.3 DictReader
- csv파일을 딕셔너리로 읽음

In [92]:
# 리스트는 csv.reader()를 썼지만 딕셔너리는 csv.DictReader() 클래스를 이용한다
with open('data/ch09_member3.csv', 'r', encoding='utf8') as f:
    dict_reader = csv.DictReader(f)
    for row in dict_reader:
        print(row)

# job 결측치가 있음. 부재시 출력하지 않겠다면?
with open('data/ch09_member3.csv', 'r', encoding='utf8') as f:
    dict_reader = csv.DictReader(f)
    dict_list = list(dict_reader)
for row in dict_reader:
    if row['job'] ==None:
        print()


{'name': '홍길동', 'age': '20', 'email': 'h@h.com', 'address': '서울시 관악구', 'job': None}
{'name': '신길동', 'age': '40', 'email': 's@h.com', 'address': '서울시 영등포구', 'job': '팀장'}
{'name': '김길동', 'age': '30', 'email': 'k@h.com', 'address': '서울시 동작구', 'job': None}


In [124]:
# header가 없는 csv파일(ch09_member1-cp.csv)을 딕셔너리로 읽기
with open('data/ch09_member1-cp.csv', encoding='cp949') as f:
    dict_reader = csv.DictReader(f, 
                                 fieldnames= ['Name', 'Age','Email','Address','Job']) # Job을 넣지 않으면 헤더 없이 
    print(list(dict_reader), end='\n')  
    # 전체출력. 여기에서 dict_reader는 파일에서 불러오는 참조타겟이다. 전체 출력시 처음부터 다 돌고 나면 빈곳을 가르키게된다
    for row in dict_reader:    # 위에서 참조가 끝났으므로 데이터를 가져오지 못한다.
        print(row)

[{'Name': '홍길동', 'Age': '20', 'Email': 'kildong@hong.com', 'Address': '서울시 관악구', 'Job': ''}, {'Name': '김길동', 'Age': '40', 'Email': 'kimdong@hong.com', 'Address': '서울시 영등포구', 'Job': '팀장'}, {'Name': '신길동', 'Age': '30', 'Email': 'sindong@hong.com', 'Address': '서울시 동작구', 'Job': ''}]


In [127]:
# 위에도 설명했지만 csv.DictReader로 생성한 리스트를 dict_reader에 할당했지만, 이는 엄연히 임시적인 참조값이라, 날아간다
# 따라서 새로운 변수를 생성해 결과값을 할당해주면 날아가지 않는다.
with open('data/ch09_member1-cp.csv', encoding='cp949') as f:
    dict_reader = csv.DictReader(f, 
                                 fieldnames= ['Name', 'Age','Email','Address','Job']) # Job을 넣지 않으면 헤더 없이 
    result = list(dict_reader)  
    print(result)
    print()
    for row in result:    
        print(row)

[{'Name': '홍길동', 'Age': '20', 'Email': 'kildong@hong.com', 'Address': '서울시 관악구', 'Job': ''}, {'Name': '김길동', 'Age': '40', 'Email': 'kimdong@hong.com', 'Address': '서울시 영등포구', 'Job': '팀장'}, {'Name': '신길동', 'Age': '30', 'Email': 'sindong@hong.com', 'Address': '서울시 동작구', 'Job': ''}]

{'Name': '홍길동', 'Age': '20', 'Email': 'kildong@hong.com', 'Address': '서울시 관악구', 'Job': ''}
{'Name': '김길동', 'Age': '40', 'Email': 'kimdong@hong.com', 'Address': '서울시 영등포구', 'Job': '팀장'}
{'Name': '신길동', 'Age': '30', 'Email': 'sindong@hong.com', 'Address': '서울시 동작구', 'Job': ''}


In [113]:
# header가 없는 csv파일(ch09_member1-cp.csv)을 딕셔너리로 읽기
with open('data/ch09_member1-cp.csv', encoding='cp949') as f:
    dict_reader = csv.DictReader(f, 
                                 fieldnames= ['Name', 'Age','Email','Address'], # Job을 넣지 않으면 헤더와 칼럼자리는 있으니 빈것으로 나옴
                                 restkey='Job')  # restkey='Job'  로 넣으면
    result = list(dict_reader)

for row in result:    
    print(row)

{'Name': '홍길동', 'Age': '20', 'Email': 'kildong@hong.com', 'Address': '서울시 관악구', 'Job': ['']}
{'Name': '김길동', 'Age': '40', 'Email': 'kimdong@hong.com', 'Address': '서울시 영등포구', 'Job': ['팀장']}
{'Name': '신길동', 'Age': '30', 'Email': 'sindong@hong.com', 'Address': '서울시 동작구', 'Job': ['']}


In [112]:

for row in result:    
    print(row)

{'Name': '홍길동', 'Age': '20', 'Email': 'kildong@hong.com', 'Address': '서울시 관악구', 'Job': ['']}
{'Name': '김길동', 'Age': '40', 'Email': 'kimdong@hong.com', 'Address': '서울시 영등포구', 'Job': ['팀장']}
{'Name': '신길동', 'Age': '30', 'Email': 'sindong@hong.com', 'Address': '서울시 동작구', 'Job': ['']}


## 3.4 DictWriter
- 딕셔너리리스트 ->csv파일

In [141]:
user1 = {'name': '홍길동', 'age': '20', 'email': 'h@h.com', 'address': '서울시 관악구'}
user2 = {'name': '신길동', 'age': '40', 'email': 's@h.com', 'address': '서울시 영등포구'}
user3 = {'name': '김길동', 'age': '30', 'email': 'k@h.com', 'address': '서울시 동작구'}
user_list = [user1, user2, user3]
field_names = list(user1.keys())
field_names
user_list

[{'name': '홍길동', 'age': '20', 'email': 'h@h.com', 'address': '서울시 관악구'},
 {'name': '신길동', 'age': '40', 'email': 's@h.com', 'address': '서울시 영등포구'},
 {'name': '김길동', 'age': '30', 'email': 'k@h.com', 'address': '서울시 동작구'}]

In [137]:
with open('data/ch09_member4.csv', 'w', newline="", encoding='utf-8') as f: # newline= 은 기본이 행바꿈
    dict_writer = csv.DictWriter(f,
                                fieldnames=field_names)
    dict_writer.writeheader()  # header 쓰기  / 위 fieldnames=field_names 에서 설정된 값을 넣는다.
    dict_writer.writerows(user_list)

## csv <-> 데이터프레임

In [139]:
import pandas as pd
member = pd.read_csv('data/ch09_member3.csv')  # 기본이 encoding='utf-8'
member

,name,age,email,address,job
0,홍길동,20,h@h.com,서울시 관악구,NaN
1,신길동,40,s@h.com,서울시 영등포구,팀장
2,김길동,30,k@h.com,서울시 동작구,NaN


In [140]:
type(member)

pandas.core.frame.DataFrame

# 4절. JSON 데이터 저장하고 불러오고(dump, load)
- 딕셔너리 리스트 <-> JSON파일 (기본)
- 객체리스트 <-> JSON파일
## 4.1 dump(파일 출력)

In [145]:
data = [{'name': '홍길동', 'age': 20, 'email': 'h@h.com', 'address': '서울시 관악구'},
         {'name': '신길동', 'age': 40, 'email': 's@h.com', 'address': '서울시 영등포구'},
         {'name': '김길동', 'age': 30, 'email': 'k@h.com', 'address': '서울시 동작구'}]

In [150]:
# ensure_ascii 매개변수
    # True  : 비 ASCII 문자는 유니코드 형태로 저장
    # False  : 비 ASCII 문자 원래 형태로 저장
import json
with open('data/ch09_member.json', 'w', encoding='utf-8') as jsonfile:
    json.dump(data,  # 딕셔너리 리스트가 기본
             jsonfile,  # 스트림 이름
             ensure_ascii=False,   # 비 아스키 코드 형태 유지
             indent='\t'  # 들여쓰기를 '  ' 만큼 할거야, 각 항목에 대한 줄바꿈이 포함되어 있다.
             )
    

In [1]:
class Member:
    def __init__(self, name, age, email, address):
        self.name = name
        self.age = age
        self.email = email
        self.address = address
    def __str__(self):
        return '{}, {}, {}, {}'.format(self.name,
                                       self.age,
                                       self.email,
                                       self.address)
    def as_dict(self):
        return {
            'name':self.name,
            'age':self.age,
            'email':self.email,
            'address':self.address
                }    
    
    def __eq__(self, other):
        if isinstance(other, Member):
            return self.__str__() == other.__str__()
#            return self.__dict__() == other.__dict__()
        else:
            return False

In [2]:
user1 = Member('홍길동', 22, 'a@a.com', '신림동')
user2 = Member('홍길동', 22, 'a@a.com', '신림동')
print(user1.__eq__('홍길동'))
print(user1==user2)

False
True


In [12]:
user_list = []
user_list.append(user1)
for row in user_list:
    print(row)

홍길동, 22, a@a.com, 신림동


In [14]:
for row in user1:
    print(row)

TypeError: 'Member' object is not iterable

In [15]:
member_list = [Member('홍길동', 22, 'a@a.com', '신림동'),
               Member('김길동', 22, 'a@a.com', '신림동'),
               Member('신길동', 22, 'a@a.com', '신림동')]


In [17]:
import json
with open('data/ch09_member1.json', 'w', encoding='utf-8') as jsonfile:
    json.dump(member_list,
             jsonfile,
             ensure_ascii=False,
             indent='\t',
             default=Member.as_dict #객체를 딕셔너리로 변경하는 함수가 필요
             )

## 4.2 load(파일입력)
- json파일 -> 딕셔너리 리스트 (기본)
- json파일 -> 객체리스트

In [18]:
def as_member(dic):
    '매개변수로 딕셔너리를 받아 Member 객체를 return'
    return Member(dic.get('name'), dic.get('age'), dic.get('email'), dic.get('address'))

In [21]:
member = as_member({'name': '홍길동', 'age': 20, 'email': 'h@h.com', 'address': '서울시 관악구'})
print(member)

홍길동, 20, h@h.com, 서울시 관악구


In [20]:
print(as_member)

<function as_member at 0x0000027765CF0280>


In [22]:
# 딕녀서너리로 로드
with open('data/ch09_member1.json', 'r', encoding='utf-8') as js:
    load_data = json.load(js)
load_data

[{'name': '홍길동', 'age': 22, 'email': 'a@a.com', 'address': '신림동'},
 {'name': '김길동', 'age': 22, 'email': 'a@a.com', 'address': '신림동'},
 {'name': '신길동', 'age': 22, 'email': 'a@a.com', 'address': '신림동'}]

In [30]:
# json 파일을 딕셔너리로 불러서 객체로 변환하여 로드
with open('data/ch09_member1.json', 'r', encoding='utf-8') as js:
    load_data = json.load(js,
                         object_hook=as_member) # object_hook= 딕셔너리를 받아 Member객체로 리턴
    for data in load_data:
        print(data)

홍길동, 22, a@a.com, 신림동
김길동, 22, a@a.com, 신림동
신길동, 22, a@a.com, 신림동


### Json -> 데이터프레임

In [27]:
import pandas as pd
pd.read_json('data/ch09_member1.json')
# pd.함수() : encoding='utf-8' 이 기본값

,name,age,email,address
0,홍길동,22,a@a.com,신림동
1,김길동,22,a@a.com,신림동
2,신길동,22,a@a.com,신림동


In [31]:
pd.read_json('data/한국토지주택공사_남양주왕숙2 토지이용계획도_20210924.json')

,image
data,/9j/4AAQSkZJRgABAQEB9AH0AAD/4RemRXhpZgAATU0AKg...
mime,image/jpeg


# 5절. 연습문제

## 실습형

In [32]:
with open('data/ch09_customs.txt', 'r', encoding='utf-8') as f:
    

SyntaxError: incomplete input (3210525131.py, line 2)